# Introduction
<p>The dataset is from <a href='https://www.kaggle.com/andrewmvd/data-analyst-jobs'> <span> Kaggle</span> </a>.</p>
<p>There are three questions of interest:</p>
<p>1.Find the best jobs by salary and company rating</p>
<p>2.Explore skills required in job descriptions</p>
<p>3.Predict salary based on industry, location, company revenue</p>

Firstly, read in the dataset.

In [122]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
%matplotlib inline

step 1: load and clean data

In [170]:
# load dataset
df = pd.read_csv('./Dataset/DataAnalyst.csv',index_col=0)

df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply
0,"Data Analyst, Center on Immigration and Justic...",$37K-$66K (Glassdoor est.),Are you eager to roll up your sleeves and harn...,3.2,Vera Institute of Justice\n3.2,"New York, NY","New York, NY",201 to 500 employees,1961,Nonprofit Organization,Social Assistance,Non-Profit,$100 to $500 million (USD),-1,True
1,Quality Data Analyst,$37K-$66K (Glassdoor est.),Overview\n\nProvides analytical and technical ...,3.8,Visiting Nurse Service of New York\n3.8,"New York, NY","New York, NY",10000+ employees,1893,Nonprofit Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),-1,-1
2,"Senior Data Analyst, Insights & Analytics Team...",$37K-$66K (Glassdoor est.),We’re looking for a Senior Data Analyst who ha...,3.4,Squarespace\n3.4,"New York, NY","New York, NY",1001 to 5000 employees,2003,Company - Private,Internet,Information Technology,Unknown / Non-Applicable,GoDaddy,-1
3,Data Analyst,$37K-$66K (Glassdoor est.),Requisition NumberRR-0001939\nRemote:Yes\nWe c...,4.1,Celerity\n4.1,"New York, NY","McLean, VA",201 to 500 employees,2002,Subsidiary or Business Segment,IT Services,Information Technology,$50 to $100 million (USD),-1,-1
4,Reporting Data Analyst,$37K-$66K (Glassdoor est.),ABOUT FANDUEL GROUP\n\nFanDuel Group is a worl...,3.9,FanDuel\n3.9,"New York, NY","New York, NY",501 to 1000 employees,2009,Company - Private,Sports & Recreation,"Arts, Entertainment & Recreation",$100 to $500 million (USD),DraftKings,True


In [171]:
# 1)get columns and rows of dataset
print(df.shape)

# 2)check null value in columns
df.info() # not exists NaN in all columns

(2253, 15)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2253 entries, 0 to 2252
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Job Title          2253 non-null   object 
 1   Salary Estimate    2253 non-null   object 
 2   Job Description    2253 non-null   object 
 3   Rating             2253 non-null   float64
 4   Company Name       2252 non-null   object 
 5   Location           2253 non-null   object 
 6   Headquarters       2253 non-null   object 
 7   Size               2253 non-null   object 
 8   Founded            2253 non-null   int64  
 9   Type of ownership  2253 non-null   object 
 10  Industry           2253 non-null   object 
 11  Sector             2253 non-null   object 
 12  Revenue            2253 non-null   object 
 13  Competitors        2253 non-null   object 
 14  Easy Apply         2253 non-null   object 
dtypes: float64(1), int64(1), object(13)
memory usage: 281.6+ KB


In [172]:
# 3)check abnormal value in columns one by one
# df.columns[0]]
print(df['Job Title'].dtype)
df['Job Title'].value_counts() # no abnormal value

object


Data Analyst                                    405
Senior Data Analyst                              90
Junior Data Analyst                              30
Business Data Analyst                            28
Sr. Data Analyst                                 21
                                               ... 
Data Analyst-Quality Improvement Services         1
Healthcare Analyst, Claims/Encounter Data         1
Data Analyst, Remote East Coast                   1
QM DATA ANALYST II, WHITTIER, FULL TIME DAYS      1
XYZ Data Analyst                                  1
Name: Job Title, Length: 1272, dtype: int64

In [187]:
# df.columns[1]
print(df['Salary Estimate'].dtype)
df['Salary Estimate'].value_counts(dropna=False) # need to repace -1 into np.nan
df['Salary Estimate'] = df[df.columns[1]].replace('-1',np.nan)
df['Salary Estimate'].value_counts(dropna=False)
df['Salary Estimate'].isnull().mean()  # only exists 1 missing value, so I decide to drop this data
df[df['Salary Estimate'].isnull()]
df.drop(index=2149,inplace=True)

object


,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply


In [174]:
# df.columns[2]
print(df['Job Description'].dtype)
df['Job Description'].value_counts(dropna=False)
df['Job Description'].isnull().mean()

object


0.0

In [179]:
# df.columns[3]
print(df['Rating'].dtype)
df['Rating'].value_counts(dropna=False)
df['Rating'] = df[df.columns[3]].replace(-1,np.nan)
df['Rating'].isnull().mean()
df['Rating'].value_counts(dropna=False)
df[df['Rating'].isnull()]   # delay cleaning

float64


,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply
11,Data Analyst,$37K-$66K (Glassdoor est.),BulbHead is currently seeking a Data Analyst t...,NaN,BulbHead,"Fairfield, NJ",-1,1 to 50 employees,-1,Company - Private,-1,-1,Unknown / Non-Applicable,-1,-1
21,Data Science Analyst,$37K-$66K (Glassdoor est.),"Job Description\nOur client, a music streaming...",NaN,MUSIC & Entertainment,"New York, NY","Marina del Rey, CA",Unknown,-1,Company - Public,-1,-1,Unknown / Non-Applicable,-1,-1
34,Data Analyst (Games),$46K-$87K (Glassdoor est.),Carry1st is the leading mobile game publisher ...,NaN,Carry1st,"New York, NY",-1,-1,-1,-1,-1,-1,-1,-1,-1
36,Data Business Analyst,$46K-$87K (Glassdoor est.),"At Clear Street, we are disrupting the institu...",NaN,Clear Street,"New York, NY","New York, NY",51 to 200 employees,2018,Company - Public,-1,-1,$1 to $5 million (USD),-1,-1
40,"Business Analyst, Data Platforms",$46K-$87K (Glassdoor est.),Company Description\n\nPinto is building the w...,NaN,Pinto,"New York, NY","New York, NY",1 to 50 employees,-1,Company - Private,-1,-1,Unknown / Non-Applicable,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2200,Data Analyst,$49K-$91K (Glassdoor est.),Role Data Analyst Duration12+ months Location ...,NaN,"TechAspect Solutions, Inc. dba TA Digital","Centennial, CO",-1,-1,-1,-1,-1,-1,-1,-1,-1
2202,Financial Data Analyst,$49K-$91K (Glassdoor est.),Position:Financial Data AnalystJob Description...,NaN,Black Knight Financial Technology Solutions,"Denver, CO",-1,-1,-1,-1,-1,-1,-1,-1,-1
2239,Senior Contract Data Analyst,$78K-$104K (Glassdoor est.),OverviewAmyx is seeking to hire a Senior Contr...,NaN,"Amyx, Iinc.","Aurora, CO",-1,-1,-1,-1,-1,-1,-1,-1,-1
2246,"Technical Business Analyst (SQL, Data analytic...",$78K-$104K (Glassdoor est.),Spiceorb is looking for Technical Business Ana...,NaN,Spiceorb,"Denver, CO",-1,-1,-1,-1,-1,-1,-1,-1,-1


In [188]:
# df.columns[4]
print(df['Company Name'].dtype)
df['Company Name'].value_counts(dropna=False)
df['Company Name'].isnull().mean()
df[df['Company Name'].isnull()]
df.drop(index=1860,inplace=True)

object


,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply


In [189]:
# df.columns[5]
print(df['Location'].dtype)
df['Location'].value_counts(dropna=False)
df['Location'].isnull().mean()

object


0.0

In [191]:
# df.columns[6]
print(df['Headquarters'].dtype)
df['Headquarters'].value_counts(dropna=False)
df['Headquarters'] = df[df.columns[6]].replace('-1',np.nan)
df['Headquarters'].isnull().mean()
df[df['Headquarters'].isnull()] # delay cleaning

object


,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply
11,Data Analyst,$37K-$66K (Glassdoor est.),BulbHead is currently seeking a Data Analyst t...,NaN,BulbHead,"Fairfield, NJ",NaN,1 to 50 employees,-1,Company - Private,-1,-1,Unknown / Non-Applicable,-1,-1
34,Data Analyst (Games),$46K-$87K (Glassdoor est.),Carry1st is the leading mobile game publisher ...,NaN,Carry1st,"New York, NY",NaN,-1,-1,-1,-1,-1,-1,-1,-1
55,Data Reporting Analyst,$46K-$87K (Glassdoor est.),OverviewThe Data Analyst is a new position in ...,NaN,NADAP NYS INC.,"New York, NY",NaN,-1,-1,-1,-1,-1,-1,-1,-1
68,Data Science Analyst,$51K-$88K (Glassdoor est.),Job Details\n\nLevel\n\nExperienced\n\nJob Loc...,NaN,Greater New York Mutual Insurance Companies (GNY),"New York, NY",NaN,-1,-1,-1,-1,-1,-1,-1,-1
90,Data Analyst,$51K-$87K (Glassdoor est.),NYU Grossman School of Medicine is one of the ...,NaN,NYU Langone Medical Center,"New York, NY",NaN,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2200,Data Analyst,$49K-$91K (Glassdoor est.),Role Data Analyst Duration12+ months Location ...,NaN,"TechAspect Solutions, Inc. dba TA Digital","Centennial, CO",NaN,-1,-1,-1,-1,-1,-1,-1,-1
2202,Financial Data Analyst,$49K-$91K (Glassdoor est.),Position:Financial Data AnalystJob Description...,NaN,Black Knight Financial Technology Solutions,"Denver, CO",NaN,-1,-1,-1,-1,-1,-1,-1,-1
2239,Senior Contract Data Analyst,$78K-$104K (Glassdoor est.),OverviewAmyx is seeking to hire a Senior Contr...,NaN,"Amyx, Iinc.","Aurora, CO",NaN,-1,-1,-1,-1,-1,-1,-1,-1
2246,"Technical Business Analyst (SQL, Data analytic...",$78K-$104K (Glassdoor est.),Spiceorb is looking for Technical Business Ana...,NaN,Spiceorb,"Denver, CO",NaN,-1,-1,-1,-1,-1,-1,-1,-1


In [193]:
# df.columns[7]
print(df['Size'].dtype)
df['Size'].value_counts(dropna=False)
df['Size'] = df[df.columns[7]].replace('-1',np.nan)
df['Size'].isnull().mean()
df[df['Size'].isnull()] # delay cleaning

object


,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply
34,Data Analyst (Games),$46K-$87K (Glassdoor est.),Carry1st is the leading mobile game publisher ...,NaN,Carry1st,"New York, NY",NaN,NaN,-1,-1,-1,-1,-1,-1,-1
55,Data Reporting Analyst,$46K-$87K (Glassdoor est.),OverviewThe Data Analyst is a new position in ...,NaN,NADAP NYS INC.,"New York, NY",NaN,NaN,-1,-1,-1,-1,-1,-1,-1
68,Data Science Analyst,$51K-$88K (Glassdoor est.),Job Details\n\nLevel\n\nExperienced\n\nJob Loc...,NaN,Greater New York Mutual Insurance Companies (GNY),"New York, NY",NaN,NaN,-1,-1,-1,-1,-1,-1,-1
90,Data Analyst,$51K-$87K (Glassdoor est.),NYU Grossman School of Medicine is one of the ...,NaN,NYU Langone Medical Center,"New York, NY",NaN,NaN,-1,-1,-1,-1,-1,-1,-1
109,Data Analyst,$51K-$87K (Glassdoor est.),"Data Analyst\n\nJersey City, NJ\n\n12+ months ...",NaN,Vertex Intel Systems,"Jersey City, NJ",NaN,NaN,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2200,Data Analyst,$49K-$91K (Glassdoor est.),Role Data Analyst Duration12+ months Location ...,NaN,"TechAspect Solutions, Inc. dba TA Digital","Centennial, CO",NaN,NaN,-1,-1,-1,-1,-1,-1,-1
2202,Financial Data Analyst,$49K-$91K (Glassdoor est.),Position:Financial Data AnalystJob Description...,NaN,Black Knight Financial Technology Solutions,"Denver, CO",NaN,NaN,-1,-1,-1,-1,-1,-1,-1
2239,Senior Contract Data Analyst,$78K-$104K (Glassdoor est.),OverviewAmyx is seeking to hire a Senior Contr...,NaN,"Amyx, Iinc.","Aurora, CO",NaN,NaN,-1,-1,-1,-1,-1,-1,-1
2246,"Technical Business Analyst (SQL, Data analytic...",$78K-$104K (Glassdoor est.),Spiceorb is looking for Technical Business Ana...,NaN,Spiceorb,"Denver, CO",NaN,NaN,-1,-1,-1,-1,-1,-1,-1


In [195]:
# df.columns[8]
print(df['Founded'].dtype)
df['Founded'].value_counts(dropna=False)
df['Founded'] = df[df.columns[8]].replace(-1,np.nan)
df['Founded'].value_counts(dropna=False)
df['Founded'].isnull().mean()
df[df['Founded'].isnull()] # delay cleaning

float64


,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply
11,Data Analyst,$37K-$66K (Glassdoor est.),BulbHead is currently seeking a Data Analyst t...,NaN,BulbHead,"Fairfield, NJ",NaN,1 to 50 employees,NaN,Company - Private,-1,-1,Unknown / Non-Applicable,-1,-1
15,Sustainability Data Analyst,$37K-$66K (Glassdoor est.),Job Description\nRole Description\n\nSustainab...,3.6,CodeGreen Solutions\n3.6,"New York, NY","New York, NY",1 to 50 employees,NaN,Company - Private,Building & Personnel Services,Business Services,Unknown / Non-Applicable,-1,-1
21,Data Science Analyst,$37K-$66K (Glassdoor est.),"Job Description\nOur client, a music streaming...",NaN,MUSIC & Entertainment,"New York, NY","Marina del Rey, CA",Unknown,NaN,Company - Public,-1,-1,Unknown / Non-Applicable,-1,-1
23,Data Analyst,$37K-$66K (Glassdoor est.),Haven Life is an insurtech innovator at MassMu...,3.5,Andiamo\n3.5,"New York, NY","Warren, MI",201 to 500 employees,NaN,Company - Private,Casual Restaurants,"Restaurants, Bars & Food Services",$1 to $5 million (USD),-1,-1
24,Entry Level / Jr. Data Analyst,$37K-$66K (Glassdoor est.),Dash Technologies is an industry leading softw...,3.8,Dash Technologies Inc\n3.8,"New York, NY","Columbus, OH",1 to 50 employees,NaN,Unknown,-1,-1,Unknown / Non-Applicable,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2244,"Data Security Analyst, Sr",$78K-$104K (Glassdoor est.),Contract Duration: 9 Months\n\nLocation: Denve...,2.5,"Avacend, Inc.\n2.5","Denver, CO","Alpharetta, GA",51 to 200 employees,NaN,Company - Private,Staffing & Outsourcing,Business Services,Unknown / Non-Applicable,-1,-1
2246,"Technical Business Analyst (SQL, Data analytic...",$78K-$104K (Glassdoor est.),Spiceorb is looking for Technical Business Ana...,NaN,Spiceorb,"Denver, CO",NaN,NaN,NaN,-1,-1,-1,-1,-1,-1
2247,Marketing/Communications - Data Analyst-Marketing,$78K-$104K (Glassdoor est.),Job Description\nJob Title: Marketing/Communic...,4.1,APN Software Services Inc.\n4.1,"Broomfield, CO","Newark, CA",51 to 200 employees,NaN,Company - Private,Computer Hardware & Software,Information Technology,$25 to $50 million (USD),-1,-1
2248,RQS - IHHA - 201900004460 -1q Data Security An...,$78K-$104K (Glassdoor est.),Maintains systems to protect data from unautho...,2.5,"Avacend, Inc.\n2.5","Denver, CO","Alpharetta, GA",51 to 200 employees,NaN,Company - Private,Staffing & Outsourcing,Business Services,Unknown / Non-Applicable,-1,-1


In [196]:
# df.columns[9]
print(df['Type of ownership'].dtype)
df['Type of ownership'].value_counts(dropna=False)
df['Type of ownership'] = df[df.columns[9]].replace('-1',np.nan)
df['Type of ownership'].isnull().mean()
df[df['Type of ownership'].isnull()] # delay cleaning

object


,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply
34,Data Analyst (Games),$46K-$87K (Glassdoor est.),Carry1st is the leading mobile game publisher ...,NaN,Carry1st,"New York, NY",NaN,NaN,NaN,NaN,-1,-1,-1,-1,-1
55,Data Reporting Analyst,$46K-$87K (Glassdoor est.),OverviewThe Data Analyst is a new position in ...,NaN,NADAP NYS INC.,"New York, NY",NaN,NaN,NaN,NaN,-1,-1,-1,-1,-1
68,Data Science Analyst,$51K-$88K (Glassdoor est.),Job Details\n\nLevel\n\nExperienced\n\nJob Loc...,NaN,Greater New York Mutual Insurance Companies (GNY),"New York, NY",NaN,NaN,NaN,NaN,-1,-1,-1,-1,-1
90,Data Analyst,$51K-$87K (Glassdoor est.),NYU Grossman School of Medicine is one of the ...,NaN,NYU Langone Medical Center,"New York, NY",NaN,NaN,NaN,NaN,-1,-1,-1,-1,-1
109,Data Analyst,$51K-$87K (Glassdoor est.),"Data Analyst\n\nJersey City, NJ\n\n12+ months ...",NaN,Vertex Intel Systems,"Jersey City, NJ",NaN,NaN,NaN,NaN,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2200,Data Analyst,$49K-$91K (Glassdoor est.),Role Data Analyst Duration12+ months Location ...,NaN,"TechAspect Solutions, Inc. dba TA Digital","Centennial, CO",NaN,NaN,NaN,NaN,-1,-1,-1,-1,-1
2202,Financial Data Analyst,$49K-$91K (Glassdoor est.),Position:Financial Data AnalystJob Description...,NaN,Black Knight Financial Technology Solutions,"Denver, CO",NaN,NaN,NaN,NaN,-1,-1,-1,-1,-1
2239,Senior Contract Data Analyst,$78K-$104K (Glassdoor est.),OverviewAmyx is seeking to hire a Senior Contr...,NaN,"Amyx, Iinc.","Aurora, CO",NaN,NaN,NaN,NaN,-1,-1,-1,-1,-1
2246,"Technical Business Analyst (SQL, Data analytic...",$78K-$104K (Glassdoor est.),Spiceorb is looking for Technical Business Ana...,NaN,Spiceorb,"Denver, CO",NaN,NaN,NaN,NaN,-1,-1,-1,-1,-1


In [142]:
# df.columns[10]
print(df['Industry'].dtype)
df['Industry'].value_counts(dropna=False)
df['Industry'] = df[df.columns[10]].replace('-1',np.nan)
df['Industry'].isnull().mean()

object


0.15667998224589436

In [143]:
# df.columns[11]
print(df[df.columns[11]].dtype)
df[df.columns[11]].value_counts(dropna=False)
df[df.columns[11]] = df[df.columns[11]].replace('-1',np.nan)
df[df.columns[11]].isnull().mean()

object


0.15667998224589436

In [144]:
# df.columns[12]
print(df[df.columns[12]].dtype)
df[df.columns[12]].value_counts(dropna=False)
df[df.columns[12]] = df[df.columns[12]].replace('-1',np.nan)
df[df.columns[12]].isnull().mean()

object


0.0723479804704838

In [148]:
# df.columns[13]
print(df[df.columns[13]].dtype)
df[df.columns[13]].value_counts(dropna=False)
df[df.columns[13]] = df[df.columns[13]].replace('-1',np.nan)
df[df.columns[13]].isnull().mean()

object


0.7687527740790058

In [149]:
# df.columns[14]
print(df[df.columns[14]].dtype)
df[df.columns[14]].value_counts()
df[df.columns[14]] = df[df.columns[14]].replace('-1',0)
df[df.columns[14]] = df[df.columns[14]].replace('True',1)

object


Job Title
0.0
Salary Estimate
0.044385264092321346
Job Description
0.0
Rating
12.072791833111406
Company Name
0.044385264092321346
Location
0.0
Headquarters
7.634265423879272
Size
7.23479804704838
Founded
29.294274300932088
Type of ownership
7.23479804704838
Industry
15.667998224589436
Sector
15.667998224589436
Revenue
7.23479804704838
Competitors
76.87527740790058
Easy Apply
0.0


In [18]:
# Salary Estimate 将数据变为高，低两列。变为int型
# Size列同上